In [1]:
import pandas as pd
import numpy as np
import math
ori = pd.read_excel(r'CS71.xlsx')
data = ori.iloc[:142,0]

xl=[]#序列
fasta=[]#序列名字
xln=[]#每个序列的长度

for i in range(len(data)):
    if i%2==0:
        a=data.iloc[i]
        fasta.append(a)
    else:
        c=data.iloc[i]
        xl.append(c)
        xln.append(len(c))

def Kmers_funct(seq, size): 
    return [seq[x:x+size] for x in range(len(seq) - size + 1)]
one_mers=[]
for i in range(len(xl)):
    myseq = xl[i]
    one_mers.append(Kmers_funct(myseq, size=1))

# 把所有连起来，成一个长序列
zimu=[]
for i in range(len(one_mers)):
    zimu.extend(one_mers[i])

In [2]:
name = ori.iloc[:202,2]
start = ori.iloc[:202,3]
l = ori.iloc[:202,4]
end =  ori.iloc[:202,5]
num = ori.iloc[:202,6].astype(np.int64)
value = ori.iloc[:202,7]

lir = {'name':name,'start':start,'l':l,'end':end,'num':num,'value':value}
df = pd.DataFrame(lir)

n = len(value)
i = 0
while i<n-1:
    index_1 = df.iloc[i, 0]
    index_2 = df.iloc[i + 1, 0]
    if index_1 == index_2:
        if df.iloc[i,5]>df.iloc[i+1,5]:
            df.iloc[i,:]=df.iloc[i+1,:]
            df.drop(index=i + 1, inplace=True)
            df.reset_index(drop=True,inplace=True)
        else:
            df.drop(index=i+1, inplace=True)
            df.reset_index(drop=True,inplace=True)
        n -= 1
    else:
        i += 1

core=[]
M=[]
for i in range(len(xl)):
    core.append((df.iloc[i,1]+df.iloc[i,3])/2)
    M.append(df.iloc[i,2])

M_2 = [M[i]/2 for i in range(len(M))]
ss = []
for i in range(len(xl)):
    a=1
    b=[]
    while a<= M_2[i]:
        b.append(1)
        a=a+1
    while a<=xln[i] and a>M_2[i]:
        b.append(a-math.floor(M_2[i]))
        a=a+1
    ss.append(b)
    
fs = []
for i in range(len(xl)):
    a=1
    b=[]
    while a<= xln[i]-M_2[i]:
        b.append(math.floor(M_2[i])+a)
        a=a+1
    while a<=xln[i] and a>xln[i]-M_2[i]:
        b.append(xln[i])
        a=a+1
    fs.append(b)
    
#拉长
SS=[]
FS=[]
for i in range(len(ss)):
    SS.extend(ss[i])
    FS.extend(fs[i])

In [3]:
ASAS_ = ori.iloc[:,1]
ASAS = []
for i in range(len(ASAS_)):
    ASAS.append(ASAS_[i])
######把长序列按序列排序分成（链数）组
xln_sum = []
for i in range(len(xln)):
    if i == 0:
        xln_sum.append(xln[i])
    else:
        xln_sum.append(sum(xln[:i+1]))
xln_sum.insert(0,0)
ASA = [ASAS[xln_sum[i]:xln_sum[i+1]] for i in range(len(xln))]

#############得到核苷酸对应的ASAS值
def value_get(one_mer,tp,t):
    get = 0
    for i in range(len(one_mer)):
        if one_mer[i] == t:
            get += tp[i]
    return get

########求在模块内部某一核苷酸的ASAS和/模块内ASAS总和
pa_asan_mk=[]
pc_asan_mk=[]
pg_asan_mk=[]
pu_asan_mk=[]

##########求在模块内部某一核苷酸的ASAS和/全ASAS总和
pa_asan_xl=[]
pc_asan_xl=[]
pg_asan_xl=[]
pu_asan_xl=[]
for i in range(len(xl)):
    for j in range(len(xl[i])):
        one_mer = one_mers[i][ss[i][j]-1:fs[i][j]]
        asa = ASA[i][ss[i][j]-1:fs[i][j]]
        
        #求在模块内部某一核苷酸的ASAS和/模块内ASAS总和
        pa_asan_mk.append(value_get(one_mer,asa,'A')/sum(ASA[i][ss[i][j]-1:fs[i][j]]))
        pc_asan_mk.append(value_get(one_mer,asa,'C')/sum(ASA[i][ss[i][j]-1:fs[i][j]]))
        pg_asan_mk.append(value_get(one_mer,asa,'G')/sum(ASA[i][ss[i][j]-1:fs[i][j]]))
        pu_asan_mk.append(value_get(one_mer,asa,'U')/sum(ASA[i][ss[i][j]-1:fs[i][j]]))
        
        #求在模块内部某一核苷酸的ASAS和/全ASAS总和
        pa_asan_xl.append(value_get(one_mer,asa,'A')/sum(ASA[i]))
        pc_asan_xl.append(value_get(one_mer,asa,'C')/sum(ASA[i]))
        pg_asan_xl.append(value_get(one_mer,asa,'G')/sum(ASA[i]))
        pu_asan_xl.append(value_get(one_mer,asa,'U')/sum(ASA[i]))
        

In [ ]:
zongjie= {'ASAS':ASAS,
          'pa_asan_mk':pa_asan_mk,
          'pc_asan_mk':pc_asan_mk,
          'pg_asan_mk':pg_asan_mk,
          'pu_asan_mk':pu_asan_mk,
          'pa_asan_xl':pa_asan_xl,
          'pc_asan_xl':pc_asan_xl,
          'pg_asan_xl':pg_asan_xl,
          'pu_asan_xl':pu_asan_xl,}

df = pd.DataFrame(zongjie)
df.to_csv("SASA.csv", index=False)

In [ ]:
-